In [1]:
import QUANTAXIS as QA
import threading
import pandas as pd

In [2]:
user = QA.QA_User(username='admin',password='940809x')
portfolio = user.new_portfolio('example')
# 创建两个account
#这里是创建一个资产组合,然后在组合里面创建两个account  你可以想象成股票里面的两个策略账户
#然后返回的是这个账户的id
a_1 = portfolio.new_account(account_cookie='a1')
a_2 = portfolio.new_account(account_cookie='a2')


In [3]:
a_1

< QA_Account a1 market: stock_cn>

In [4]:
"""
然后这里 是创建一个交易前置  你可以理解成 创建了一个无界面的通达信客户端
然后start()开启这个客户端
连接到backtest的broker上 这个broker可以更换
"""
# 创建一个交易前置
market = QA.QA_Market()
# 交易前置连接broker 
market.start()
market.connect(QA.RUNNING_ENVIRONMENT.BACKETEST)

# 打印market
print(market)



{'worker': < QA_Worker market_event id = 1766490941648 >, 'event': < QA_Event query_order None False , id = 1766490942264 >, 'res': None, 'callback': False, 'task_id': 'Task_evbcmD20', 'engine': 'ORDER'}
receive tasks
<QA_Market with ['backtest'] QA_Broker >


In [5]:

"""
登陆到这个交易前置上 把你刚才的两个账户
"""
# 登陆交易
market.login(QA.BROKER_TYPE.BACKETEST,a_1.account_cookie, a_1)
market.login(QA.BROKER_TYPE.BACKETEST,a_2.account_cookie, a_2)
# 打印市场中的交易账户
print(market.get_account_cookie())


subscribe
subscribe
['a1', 'a2']


In [6]:
#然后这里 往交易前置里面添加订单 这个操作是异步的

In [7]:
market.insert_order(account_cookie=a_1.account_cookie, money=100000, amount=None,price=None, amount_model=QA.AMOUNT_MODEL.BY_MONEY,time='2017-12-01', code='600010', 
                    order_model=QA.ORDER_MODEL.CLOSE, towards=QA.ORDER_DIRECTION.BUY,market_type=QA.MARKET_TYPE.STOCK_CN,
                   frequence=QA.FREQUENCE.DAY,broker_name=QA.BROKER_TYPE.BACKETEST)

None by_money 2017-12-01 600010 2.62 CLOSE 1 100000
{'worker': < QA_Worker market_event id = 1766490941648 >, 'event': < QA_Event receive_order None <bound method QA_Market.on_insert_order of <QA_Market with ['backtest'] QA_Broker >> , id = 1766491915096 >, 'res': None, 'callback': False, 'task_id': 'Task_aCeTp9RS', 'engine': 'ORDER'}
receive tasks


In [8]:
market.insert_order(account_cookie=a_1.account_cookie, amount=100,price=None, amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,time='2017-12-01', code='000001', 
                    order_model=QA.ORDER_MODEL.CLOSE, towards=QA.ORDER_DIRECTION.BUY,market_type=QA.MARKET_TYPE.STOCK_CN,
                   frequence=QA.FREQUENCE.DAY,broker_name=QA.BROKER_TYPE.BACKETEST)

100 on_insert_orderby_amount
 < QA_Order realorder_id Order_ecHqxVwg datetime:2017-12-01 15:00:00 code:600010 amount:38100 price:2.62 towards:1 btype:stock_cn order_id:Order_ecHqxVwg account:a1 status:queued >2017-12-01
 queued000001
 13.0 CLOSE 1 None
{'worker': < QA_Worker market_event id = 1766490941648 >, 'event': < QA_Event receive_order None <bound method QA_Market.on_insert_order of <QA_Market with ['backtest'] QA_Broker >> , id = 1766491928616 >, 'res': None, 'callback': False, 'task_id': 'Task_RIXda7tM', 'engine': 'ORDER'}
receive tasks


In [9]:
#
"""
下单以后 现金不会减少 但是可用现金会被扣除
因为如果是市价单 你的成交价未定
没法直接减少现金
可用现金减少 cash不减少 等到settle 等到成功交易的时候 才会扣cash

"""

'\n下单以后 现金不会减少 但是可用现金会被扣除\n因为如果是市价单 你的成交价未定\n没法直接减少现金\n可用现金减少 cash不减少 等到settle 等到成功交易的时候 才会扣cash\n\n'

In [10]:
market.session[a_1.account_cookie].cash_available

898699.675

In [11]:
market.session[a_1.account_cookie].cash

[1000000]

In [12]:
"""
这里是交易前置内部的订单队列
"""


'\n这里是交易前置内部的订单队列\n'

on_insert_order
< QA_Order realorder_id Order_RQyS5C2v datetime:2017-12-01 15:00:00 code:000001 amount:100 price:13.0 towards:1 btype:stock_cn order_id:Order_RQyS5C2v account:a1 status:queued >


In [13]:
market.order_handler.order_queue()





queued


{'Order_ecHqxVwg': < QA_Order realorder_id Order_ecHqxVwg datetime:2017-12-01 15:00:00 code:600010 amount:38100 price:2.62 towards:1 btype:stock_cn order_id:Order_ecHqxVwg account:a1 status:queued >,
 'Order_RQyS5C2v': < QA_Order realorder_id Order_RQyS5C2v datetime:2017-12-01 15:00:00 code:000001 amount:100 price:13.0 towards:1 btype:stock_cn order_id:Order_RQyS5C2v account:a1 status:queued >}

In [14]:
market.order_handler.order_queue.order_list

{'Order_ecHqxVwg': < QA_Order realorder_id Order_ecHqxVwg datetime:2017-12-01 15:00:00 code:600010 amount:38100 price:2.62 towards:1 btype:stock_cn order_id:Order_ecHqxVwg account:a1 status:queued >,
 'Order_RQyS5C2v': < QA_Order realorder_id Order_RQyS5C2v datetime:2017-12-01 15:00:00 code:000001 amount:100 price:13.0 towards:1 btype:stock_cn order_id:Order_RQyS5C2v account:a1 status:queued >}

In [15]:
#pending 是指的待成交列表


In [16]:
market.order_handler.order_queue.pending

[]

In [17]:
"""
这个_trade是一个私有方法 只有模拟盘和回测才会有 实盘就是真的交易了 
这个_trade是backtest类去调用的

"""
#market._trade(QA.QA_Event(broker_name=QA.BROKER_TYPE.BACKETEST,after_success=None))

'\n这个_trade是一个私有方法 只有模拟盘和回测才会有 实盘就是真的交易了 \n这个_trade是backtest类去调用的\n\n'

In [18]:
"""下面这两个是 查询  一个是异步查询 一个是同步的(no_wait)
异步不会阻塞当前线程 同步会阻塞"""

'下面这两个是 查询  一个是异步查询 一个是同步的(no_wait)\n异步不会阻塞当前线程 同步会阻塞'

In [19]:
market.query_data(broker_name=QA.BROKER_TYPE.BACKETEST,frequence=QA.FREQUENCE.DAY,market_type=QA.MARKET_TYPE.STOCK_CN,
                 code='600010',start='2017-12-01')

{'worker': < QA_Broker backtest thread 12956 >, 'event': < QA_Event query_data None <bound method QA_Market.on_query_data of <QA_Market with ['backtest'] QA_Broker >> , id = 1766491981136 >, 'res': None, 'callback': False, 'task_id': 'Task_5BxLAEUK', 'engine': 'backtest'}
receive tasks
ON QUERY
[{'code': '600010', 'open': 2.6, 'high': 2.63, 'low': 2.56, 'close': 2.62, 'volume': 920022.0, 'amount': 239518192.0, 'date': '2017-12-01 00:00:00'}]

In [20]:
market.query_data_no_wait(broker_name=QA.BROKER_TYPE.BACKETEST,frequence=QA.FREQUENCE.DAY,market_type=QA.MARKET_TYPE.STOCK_CN,
                 code='000001',start='2017-12-14')

[{'code': '000001',
  'open': 13.15,
  'high': 13.31,
  'low': 12.91,
  'close': 13.0,
  'volume': 1001998.0,
  'amount': 1311131904.0,
  'date': '2017-12-14 00:00:00'}]

In [21]:
"""成交了以后 你可以看到账户的资产变化了"""
market.session[a_1.account_cookie]

< QA_Account a1 market: stock_cn>

In [22]:
"""待成交列表被清空"""


'待成交列表被清空'

In [23]:
market.order_handler.order_queue.pending

[]

In [24]:
"""待成交队列清空"""


'待成交队列清空'

In [25]:
market.order_handler.order_queue.order_list

{'Order_ecHqxVwg': < QA_Order realorder_id Order_ecHqxVwg datetime:2017-12-01 15:00:00 code:600010 amount:38100 price:2.62 towards:1 btype:stock_cn order_id:Order_ecHqxVwg account:a1 status:success_all >,
 'Order_RQyS5C2v': < QA_Order realorder_id Order_RQyS5C2v datetime:2017-12-01 15:00:00 code:000001 amount:100 price:13.0 towards:1 btype:stock_cn order_id:Order_RQyS5C2v account:a1 status:success_all >}

In [26]:
market.order_handler.order_queue()

{'Order_ecHqxVwg': < QA_Order realorder_id Order_ecHqxVwg datetime:2017-12-01 15:00:00 code:600010 amount:38100 price:2.62 towards:1 btype:stock_cn order_id:Order_ecHqxVwg account:a1 status:success_all >,
 'Order_RQyS5C2v': < QA_Order realorder_id Order_RQyS5C2v datetime:2017-12-01 15:00:00 code:000001 amount:100 price:13.0 towards:1 btype:stock_cn order_id:Order_RQyS5C2v account:a1 status:success_all >}

In [27]:
"""
cash 现金减少
"""
market.session[a_1.account_cookie].cash

[1000000, 900153.0445, 898848.0445]

In [28]:
"""
因为没有触发每日结算时间 在T+1的市场 即使买入了也没有可卖的
"""
market.session[a_1.account_cookie].sell_available

Series([], Name: amount, dtype: float64)

In [29]:
sa=market.session[a_1.account_cookie].sell_available

In [30]:
ac=market.session[a_1.account_cookie]

In [31]:
ac.hold

code
000001      100
600010    38100
Name: amount, dtype: int64

In [32]:
sa

Series([], Name: amount, dtype: float64)

In [33]:
market.session[a_1.account_cookie].history

[['2017-12-01 00:00:00',
  '600010',
  2.62,
  38100,
  900153.0445,
  'Order_ecHqxVwg',
  'Order_ecHqxVwg',
  'Trade_XUE7pjI2',
  'a1',
  24.9555,
  0,
  None,
  0],
 ['2017-12-01 00:00:00',
  '000001',
  13.0,
  100,
  898848.0445,
  'Order_RQyS5C2v',
  'Order_RQyS5C2v',
  'Trade_4uoOCHic',
  'a1',
  5,
  0,
  None,
  0]]

In [34]:
"""
持仓表增加
"""
market.session[a_1.account_cookie].hold

code
000001      100
600010    38100
Name: amount, dtype: int64

In [35]:
"""
账户信息

可以看到 减少的资产 主要是因为收了手续费
"""
market.session[a_1.account_cookie].message

{'source': 'account',
 'account_cookie': 'a1',
 'portfolio_cookie': 'example',
 'user_cookie': 'USER_wp6OZvjM',
 'broker': 'backtest',
 'market_type': 'stock_cn',
 'strategy_name': None,
 'current_time': 'None',
 'allow_sellopen': False,
 'allow_t0': False,
 'margin_level': {},
 'init_assets': {'cash': 1000000, 'hold': {}},
 'init_cash': 1000000,
 'init_hold': {},
 'commission_coeff': 0.00025,
 'tax_coeff': 0.001,
 'cash': [1000000, 900153.0445, 898848.0445],
 'history': [['2017-12-01 00:00:00',
   '600010',
   2.62,
   38100,
   900153.0445,
   'Order_ecHqxVwg',
   'Order_ecHqxVwg',
   'Trade_XUE7pjI2',
   'a1',
   24.9555,
   0,
   None,
   0],
  ['2017-12-01 00:00:00',
   '000001',
   13.0,
   100,
   898848.0445,
   'Order_RQyS5C2v',
   'Order_RQyS5C2v',
   'Trade_4uoOCHic',
   'a1',
   5,
   0,
   None,
   0]],
 'trade_index': ['2017-12-01 00:00:00', '2017-12-01 00:00:00'],
 'running_time': '2019-02-05 18:41:05.250531',
 'quantaxis_version': '1.3.0.dev3',
 'running_environment': '

In [36]:
"""结算事件"""
market._settle(QA.BROKER_TYPE.BACKETEST)

{'worker': < QA_Account a1 market: stock_cn>, 'event': < QA_Event account_settle None False , id = 1766491915096 >, 'res': None, 'callback': False, 'task_id': 'Task_s8oSeczT', 'engine': 'backtest'}
receive tasks
{'worker': < QA_Account a2 market: stock_cn>, 'event': < QA_Event account_settle None False , id = 1766492196936 >, 'res': None, 'callback': False, 'task_id': 'Task_sR2I3ZMX', 'engine': 'backtest'}
receive tasks
{'worker': < QA_Broker backtest thread 12956 >, 'event': < QA_Event broker_settle None False , id = 1766492199736 >, 'res': None, 'callback': False, 'task_id': 'Task_DX3xul7R', 'engine': 'backtest'}
receive tasks
{'worker': < QA_Worker market_event id = 1766490941648 >, 'event': < QA_Event broker_settle None False , id = 1766492200520 >, 'res': None, 'callback': False, 'task_id': 'Task_FxgfQ4GK', 'engine': 'ORDER'}
receive tasks
===== SETTLED None =====
account_settleSETTLE ORDERHANDLER



In [37]:
"""
结算完以后 可卖数量就会变成和持仓数一样
"""


'\n结算完以后 可卖数量就会变成和持仓数一样\n'

orderahndle: trade
{< QA_Account a1 market: stock_cn>: < QA_Broker backtest thread 12892 >, < QA_Account a2 market: stock_cn>: < QA_Broker backtest thread 12892 >}
account_settle

In [38]:
market.session[a_1.account_cookie].hold

code
000001      100
600010    38100
Name: amount, dtype: int64

broker settle

In [39]:
market.session[a_1.account_cookie].sell_available

code
000001      100
600010    38100
Name: amount, dtype: int64

In [40]:
"""
结算完以后 待成交队列也被清空
"""



'\n结算完以后 待成交队列也被清空\n'

In [41]:
market.order_handler.order_queue()

{}

In [42]:
market.insert_order(account_cookie=a_1.account_cookie, amount=market.session[a_1.account_cookie].sell_available.get('600010',0),price=None, amount_model=QA.AMOUNT_MODEL.BY_AMOUNT,time='2017-12-05', code='600010', 
                    order_model=QA.ORDER_MODEL.CLOSE, towards=QA.ORDER_DIRECTION.SELL,market_type=QA.MARKET_TYPE.STOCK_CN,
                   frequence=QA.FREQUENCE.DAY,broker_name=QA.BROKER_TYPE.BACKETEST)

38100 by_amount 2017-12-05 600010 2.55 CLOSE -1 None
{'worker': < QA_Worker market_event id = 1766490941648 >, 'event': < QA_Event receive_order None <bound method QA_Market.on_insert_order of <QA_Market with ['backtest'] QA_Broker >> , id = 1766492298488 >, 'res': None, 'callback': False, 'task_id': 'Task_D38SOwRm', 'engine': 'ORDER'}
receive tasks


In [43]:
market.order_handler.order_queue.order_list


{}

In [44]:
market.order_handler.order_queue()

{}

In [45]:
market.session[a_1.account_cookie].sell_available

code
000001    100
600010      0
Name: amount, dtype: int64

In [46]:
#market._trade(QA.QA_Event(broker_name=QA.BROKER_TYPE.BACKETEST,after_success=None))

In [47]:
market.session[a_1.account_cookie].cash

[1000000, 900153.0445, 898848.0445]

In [48]:
market.session[a_1.account_cookie].history

[['2017-12-01 00:00:00',
  '600010',
  2.62,
  38100,
  900153.0445,
  'Order_ecHqxVwg',
  'Order_ecHqxVwg',
  'Trade_XUE7pjI2',
  'a1',
  24.9555,
  0,
  None,
  0],
 ['2017-12-01 00:00:00',
  '000001',
  13.0,
  100,
  898848.0445,
  'Order_RQyS5C2v',
  'Order_RQyS5C2v',
  'Trade_4uoOCHic',
  'a1',
  5,
  0,
  None,
  0]]

In [49]:
user.save()

account a1 save
account a2 save
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
QAACCOUNT: THIS ACCOUNT DOESNOT HAVE ANY TRADE
